In [1]:
import argparse
from os import path

import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F

from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification
from custom_model import RBERT
import pandas as pd
from tqdm import tqdm


from utils import *

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large')

helper = DataHelper(data_dir='/opt/ml/dataset/test/preprocess_test.csv',mode='inference')
_test_data = helper.from_idxs()
test_data = helper.entity_tokenize(data=_test_data, tokenizer=tokenizer)
test_dataset = RelationExtractionDataset(test_data)

probs = []

In [3]:
tokenizer.tokenize('ㄱ')

['ㄱ']

In [16]:
model_config = AutoConfig.from_pretrained('klue/roberta-large', num_labels=30)
model = AutoModelForSequenceClassification.from_pretrained('klue/roberta-large')
dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)
preds, probs = [], []
model = RBERT.from_pretrained('/opt/ml/klue-level2-nlp-08/test_entitiy_masking/best_model/'+str(1)+'_fold/',config=model_config, model_name='klue/roberta-large', dropout_rate=0.1)

    

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifi

In [17]:
model.label_classifier.linear.weight

Parameter containing:
tensor([[-0.0027, -0.0048, -0.0126,  ..., -0.0080,  0.0093, -0.0036],
        [-0.0161,  0.0107, -0.0102,  ..., -0.0055, -0.0086,  0.0049],
        [ 0.0041,  0.0151,  0.0111,  ...,  0.0086, -0.0007,  0.0061],
        ...,
        [ 0.0093,  0.0109,  0.0060,  ..., -0.0141, -0.0094,  0.0151],
        [-0.0002, -0.0090, -0.0190,  ...,  0.0167, -0.0093, -0.0182],
        [-0.0101,  0.0039,  0.0051,  ...,  0.0073,  0.0149, -0.0024]],
       requires_grad=True)

In [18]:
model = RBERT.from_pretrained('/opt/ml/klue-level2-nlp-08/test_entitiy_masking/best_model/'+str(1)+'_fold/',config=model_config, model_name='klue/roberta-large', dropout_rate=0.1)


Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

In [19]:
model.label_classifier.linear.weight

Parameter containing:
tensor([[-0.0027, -0.0048, -0.0126,  ..., -0.0080,  0.0093, -0.0036],
        [-0.0161,  0.0107, -0.0102,  ..., -0.0055, -0.0086,  0.0049],
        [ 0.0041,  0.0151,  0.0111,  ...,  0.0086, -0.0007,  0.0061],
        ...,
        [ 0.0093,  0.0109,  0.0060,  ..., -0.0141, -0.0094,  0.0151],
        [-0.0002, -0.0090, -0.0190,  ...,  0.0167, -0.0093, -0.0182],
        [-0.0101,  0.0039,  0.0051,  ...,  0.0073,  0.0149, -0.0024]],
       requires_grad=True)

In [8]:
test_dataset[0]
next(iter(dataloader))['input_ids']

[tensor([0, 0, 0, 0]),
 tensor([3625, 3784, 3995,   36]),
 tensor([ 3749, 21154,    16,    14]),
 tensor([2210,    7, 5006, 3611]),
 tensor([    7,    14, 16576,    14]),
 tensor([  14, 3611, 2283, 1543]),
 tensor([3971,   14, 1498, 2335]),
 tensor([  14, 8354, 4347, 2390]),
 tensor([ 5633, 24141,  2200,    36]),
 tensor([   7, 1878, 1891, 1497]),
 tensor([  36, 2557, 5633,  648]),
 tensor([   14,     7, 10914,  2431]),
 tensor([ 3971,  1503, 16460,  2145]),
 tensor([  14, 3934,  543, 3848]),
 tensor([11428,  2073,    36, 29128]),
 tensor([26648,  1165,    14,  2200]),
 tensor([27967,  2079,  9384, 10207]),
 tensor([  36, 6717,   14,  594]),
 tensor([ 543, 2098, 7394, 2447]),
 tensor([ 1504, 30698,  2440,  2205]),
 tensor([3669, 6104,   36, 2307]),
 tensor([2069,   86,   26, 1327]),
 tensor([4256, 8164,   18, 2073]),
 tensor([ 2371,  4586,  3912, 11894]),
 tensor([ 2062,  2052, 10914,  3926]),
 tensor([  18, 2241, 2216, 2088]),
 tensor([   2,  936, 6233,  648]),
 tensor([   1, 6233, 62

In [13]:
for data in tqdm(dataloader):
    batch = {k: v.to(device) for k, v in data.items()}

  0%|          | 0/3883 [00:00<?, ?it/s]


AttributeError: 'list' object has no attribute 'to'